In [1]:
from dbanalysis import stop_tools

## Trying some different models on a single stop

In [2]:
df = stop_tools.random_stop_data()

In [3]:
from sklearn.ensemble import RandomForestRegressor as rf

In [4]:
df.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'date',
       'hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb', 'dt'],
      dtype='object')

In [5]:
df['month']=df['dt'].dt.month
train = df[df['month']<5]
test=df[df['month']>=5]

In [6]:
train.shape

(1272, 27)

In [7]:
test.shape

(413, 27)

In [8]:
train = train.dropna()

In [9]:
train.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'date',
       'hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb', 'dt',
       'month'],
      dtype='object')

In [10]:
df['day']=df['dt'].dt.dayofweek

In [11]:
regr = rf(max_depth=100).fit(train[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']],train['traveltime'])

KeyError: "['day'] not in index"

In [ ]:
preds = regr.predict(test[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']])

In [ ]:
from sklearn import metrics

In [ ]:
metrics.mean_absolute_error(test['traveltime'],preds)

In [ ]:
metrics.r2_score(test['traveltime'],preds)

In [ ]:
import numpy as np
msk = np.random.rand(len(df))<0.8

In [ ]:
df=df.dropna()
train = df[msk]
test = df[~msk]

In [ ]:
regr = rf(max_depth=100).fit(train[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']],train['traveltime'])
preds = regr.predict(test[['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb']])
metrics.mean_absolute_error(test['traveltime'],preds)

In [ ]:
df['act2']=df['actualtime_arr_from']**2
df['act3']=df['actualtime_arr_from']**3
df['rain2']=df['rain']**2
df['rain3']=df['rain']**3
df['day2']=df['day']**2
df['day3']=df['day']**3

In [ ]:
features=['month','hour','day','actualtime_arr_from','actualtime_dep_from','rain','rhum','temp','msl','wetb','act2','act3','rain2','rain3','day2','day3']

In [ ]:
regr = rf(max_depth=100).fit(train[features],train['traveltime'])
preds = regr.predict(test[features])
metrics.mean_absolute_error(test['traveltime'],preds)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regr = LinearRegression(fit_intercept=True).fit(train[features],train['traveltime'])
preds=regr.predict(test[features])
metrics.mean_absolute_error(test['traveltime'],preds)

In [ ]:
from sklearn.neural_network import MLPRegressor as ml

In [ ]:
regr = ml().fit(train[features],train['actualtime_arr_to'])

In [ ]:
preds=regr.predict(test[features])

In [ ]:
metrics.mean_absolute_error(test['actualtime_arr_to'],preds)

In [ ]:
features=['month',
 'hour',
 'day',
 'actualtime_arr_from',
 'actualtime_dep_from',
 'rain',
 'rhum',
 'temp',
 'msl',
 'wetb']

In [ ]:
from sklearn import svm

In [ ]:
regr =svm.SVR().fit(train[features],train['traveltime'])

In [ ]:
preds=regr.predict(test[features])

In [ ]:
import pandas as pd

In [ ]:
#dumb method
preds = pd.Series([test['traveltime'].mean() for i in range(len(test))])

In [ ]:
metrics.mean_absolute_error(test['traveltime'],preds)

### The point here then is that predicting the mean isn't really so bad

In [39]:
import json
routes = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/trimmed_routes.json','r').read())

eh fuck this


# Making a singular gigantic model of a route

In [40]:
r=routes['15'][1]

In [41]:
from dbanalysis import stop_tools
import pandas as pd

In [42]:
to_concat=[]
for i in range(1, len(r)-1):
    a=r[i]
    b=r[i+1]
    df=stop_tools.get_stop_link(a,b,merge_weather=True)
    to_concat.append(df)
    
df = pd.concat(to_concat,axis=0)
    

In [43]:
df=df.dropna()

In [ ]:
df.shape

## adding the approx distances from the first stop

In [44]:
s_getter =stop_tools.stop_getter()
total_distance = 0
route_distances = {r[1]:0}
for i in range(1, len(r)-1):
    distance = s_getter.get_stop_distance(str(r[i]),str(r[i+1]))
    
    total_distance += distance
    route_distances[r[i+1]]=total_distance
    

In [ ]:
route_distances

In [45]:
df['distance']=df['stopA'].apply(lambda x: route_distances[x])

## hacking in the time of leaving the first stop

In [46]:
keys= df[df['stopA']==6318]

In [ ]:
len(keys)

In [47]:
to_concat = []
## no this doesn't work
#for row in keys.itertuples():

  #  
 #   temp_df = df[(df['dayofservice']==row[1]) & (df['tripid']==row[2])]
   # temp_df['base_time_dep'] = row[5]
#    to_concat.append(temp_df)


keys['base_time_dep']=keys['actualtime_arr_from']
keys2=keys[['tripid','dayofservice','base_time_dep']]
full_df = pd.merge(df,keys2,on=['dayofservice','tripid'])
    

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
full_df

In [ ]:
full_df['base_time_dep'].max()

In [ ]:
full_df.shape

In [ ]:
# we seem to have lost a lot of samples unfortunately, by virtue of following this process. Maybe we need to like ... use the scheduled departure times instead?

In [ ]:
full_df.columns

In [48]:
full_df['traveltime']=full_df['actualtime_arr_from']-full_df['base_time_dep']

In [ ]:
full_df

## Splitting and adding date time information

In [51]:
import numpy as np
msk = np.random.rand(len(full_df))<0.8
test = full_df[msk]

train = full_df[~msk]

In [49]:
full_df=full_df.dropna()

In [50]:
time_format = "%d-%b-%y %H:%M:%S"
full_df['dt']=pd.to_datetime(full_df['dayofservice'],format=time_format)
full_df['dayofweek']=full_df['dt'].dt.dayofweek
full_df['month']=full_df['dt'].dt.month
full_df['weekend']=full_df['dayofweek']>4

In [63]:
from sklearn.linear_model import LinearRegression
features=['hour','weekend','month','dayofweek','rhum','rain','vappr','wetb','distance','base_time_dep','msl','temp']
regr=LinearRegression(fit_intercept=True).fit(train[features],train['actualtime_arr_from'])

In [64]:
predicts=regr.predict(test[features])
print(metrics.mean_absolute_error(test['actualtime_arr_from'],predicts))
print(metrics.r2_score(test['actualtime_arr_from'],predicts))

326.73161901369065
0.9993193909641614


In [65]:
full_df['weekend']=full_df['dayofweek']>4
features=['hour','weekend','month','dayofweek','rhum','rain','vappr','wetb','distance','base_time_dep','msl','temp']
regr=LinearRegression(fit_intercept=True).fit(train[features],train['traveltime'])

In [66]:
predicts=regr.predict(test[features])
print(metrics.mean_absolute_error(test['traveltime'],predicts))
print(metrics.r2_score(test['traveltime'],predicts))

326.7316190136915
0.9278256914020145


# Model works, but there is clearly something bullshit about these r2 scores

... Testing again, but this time only on a single variable (the time taken to get from the first stop to the last stop)

In [67]:
test2 = full_df[full_df['distance']==22.60855934258725]

In [68]:
len(test2)

6593

In [69]:
predicts=regr.predict(test2[features])
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

547.6833167352945
0.21055628133687754


## As you can see

# a) The r2 score has collapsed now that we're testing only a singular variable

# b) The mean absolute error has gone up - we are about 10 minutes off in our predictions for how long it takes the bus to travel the entirety of the route

## Over all, I'm going to say that the only gains from making 'whole route models' are going to be in terms of optimizing our compute process, not in terms of accuracy.

## The two step dwelltime/traveltime modelling process still makes way more sense I reckon

With this gigantic route model approach we're:
1. Forced to drop a lot of relevant data from our analysis (the links that are shared by multiple routes)
2. Trying to approximate a horrendously complicated, messy, non linear function.
3. Unable to adequately separate the prediction of dwell time and the prediction of travel time.





# With fake neural network

In [ ]:
regr = ml().fit(train[features],train['traveltime'])

In [ ]:
predicts=regr.predict(test2[features])
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

## It also takes a shameful amount of time to train a neural network model on this ... 


### And we only save on 5 seconds

# With  svm

In [37]:
from sklearn import svm

In [28]:
regr = svm.SVR().fit(train[features],train['traveltime'])


In [29]:
import pickle
with open('svm.pickle','wb') as handle:
    pickle.dump(regr,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
import pickle
with open('svm.pickle','rb') as handle:
    regr = pickle.load(handle)

In [ ]:
from sklearn import metrics
print('### All stops ###')
predicts=regr.predict(test[features])
print(metrics.mean_absolute_error(test['traveltime'],predicts))
print(metrics.r2_score(test['traveltime'],predicts))
predicts=regr.predict(test2[features])
print('\n\n ### One Stop ###')
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

### All stops ###


In [54]:
predicts=regr.predict(test2[features])
from sklearn import metrics
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

2591.2068976663527
-11.535763697371335


# Forget about training time - svm seems to take forever to predict?

Also the results are crap. Clearly will haven to understand what this model actually does, before fine tuning it.

## With random forest

In [56]:
from sklearn.ensemble import RandomForestRegressor as rf

In [95]:
regr = rf().fit(train[features],train['traveltime'])

In [96]:
test2 = test[test['distance']==22.60855934258725]
predicts=regr.predict(test[features])

from sklearn import metrics
print(metrics.mean_absolute_error(test['traveltime'],predicts))
print(metrics.r2_score(test['traveltime'],predicts))

158.13037644084906
0.9795461178340256


In [97]:
predicts=regr.predict(test2[features])

from sklearn import metrics
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

269.99855513307983
0.7703147250617991


# This seems to be the best candidate so far

Large reduction in error from what we saw while using linear and neural networks.

Why not cluster the data then to find the best candidates for this kind of thing? Somehow minimize the size of the models through sampling important data or something. Yeah. That kind of thing.


# With k nearest neighbors

In [76]:
from sklearn.neighbors import KNeighborsRegressor

In [87]:
regr = KNeighborsRegressor(1).fit(train[features],train['traveltime'])
predicts=regr.predict(test2[features])
print(metrics.mean_absolute_error(test2['traveltime'],predicts))
print(metrics.r2_score(test2['traveltime'],predicts))

312.9153992395437
0.6483993230964855


#### Interestingly, upping the number of neighbors seems to push the results into negative territory

In [ ]:
print('hello')

# All that said, the biggest problem with the big route model is that we're dropping so much data

### How to use the rows of data that come from different routes? Is there a legitimate way to use them even?